In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy import wcs

In [ ]:
# import reprojected fits files
dust = fits.open('dust_reproject.fits')
co = fits.open('co_reproject.fits')
HI = fits.open('HI_reproject.fits')

# Mass Surface Density (M_sun/pc^-2) with log scale conversion
dust_data = 10**(dust[4].data) / 3.2

# Intensity (Kkm/s)
co_data = co[0].data

# Column density (cm^-2 Kkm/s)
HI_data = HI[0].data 

#use the dust header
hdr = dust.header

In [ ]:
NH_data = dust_data / DGR 

In [ ]:
# For if you want to do a 3-sigma cut

# Mask co_data where it's less than 0.5
co_mask = co_data >= 0.5  # 0.3 from 3-sigma cut

# Mask HI_data where it's less than 30e18
hi_mask = HI_data >= 36e18  # 36e18 from 3-sigma cut stddev = 10 from intensity then scaled 1e18 when converted to column density

# Appy masks to data
co_data_snrcut = np.where(co_mask, co_data, np.nan) # Mask out invalid co data
HI_data_snrcut = np.where(hi_mask, HI[0].data, np.nan)  # Mask out invalid HI data
NH_data_snrcut = np.where(co_mask, NH_data, np.nan) # mask out invalid gas data

In [ ]:
# conversion from co(2-1) to co(1-0)
co_10 = co_data_snrcut * 0.8
# dust derived H2 and co traced H2
total_h2 = (NH_data - HI_data_snrcut) / 2
co_derived_h2 = co_10 * xfactor
# co dark map
co_dark_h2 = total_h2 - co_derived_h2

In [ ]:
plt.style.use('dark_background')
ax1 = plt.subplot(1,1,1, projection = WCS(hdr))

# Display the first data set
im1 = ax1.imshow(co_dark_h2[600:1250, 500:1000], cmap = 'inferno')
cbar = plt.colorbar(im1, ax=ax1)  # Add color bar for the first subplot
ax1.set_xlabel('Right Ascension', fontsize = 14)
ax1.set_ylabel('Declination', fontsize = 14)
ax1.set_title('"CO-dark" H2 Map', fontsize = 14)
ax1.coords.grid(False)
cbar.set_label(r'$N(\mathrm{H})$ cm$^{-2}$',fontsize = 14)